# Analyze SATURN Outputs

In [1]:
import scanpy as sc

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/scanpy/_utils/__init__.py:35: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/anndata/__init__.py:70: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/anndata/__init__.py:70: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/anndata/__init__.py:70: FutureWarning: Imp

In [2]:
chicken_heart_saturn_ad = sc.read("export_data/test256_data_chicken_heart_uce_adata_tabula_sapiens_heart_org_saturn_seed_0.h5ad")
chicken_heart_saturn_ad

AnnData object with n_obs × n_vars = 33820 × 256
    obs: 'labels', 'labels2', 'ref_labels', 'species'
    uns: 'labels_colors', 'neighbors', 'pca', 'species_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'macrogenes'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [3]:
nmr_spleen_saturn_ad = sc.read("export_data/test256_data_tabula_sapiens_spleen_nmr_female_uce_adata_org_saturn_seed_0.h5ad")
nmr_spleen_saturn_ad

AnnData object with n_obs × n_vars = 75058 × 256
    obs: 'labels', 'labels2', 'ref_labels', 'species'
    obsm: 'macrogenes'

In [4]:
green_monkey_lung_saturn_ad = sc.read("export_data/test256_data_green_monkey_covid_uce_adata_tabula_sapiens_lung_org_saturn_seed_0.h5ad")
green_monkey_lung_saturn_ad

AnnData object with n_obs × n_vars = 83977 × 256
    obs: 'labels', 'labels2', 'ref_labels', 'species'
    obsm: 'macrogenes'

In [5]:
chicken_retina_saturn_ad = sc.read("export_data/test256_data_chicken_uce_adata_human_retina_org_saturn_seed_0.h5ad")
chicken_retina_saturn_ad

AnnData object with n_obs × n_vars = 134401 × 256
    obs: 'labels', 'labels2', 'ref_labels', 'species'
    obsm: 'macrogenes'

# Produce Cell Type Maps

In [6]:
import pandas as pd
import numpy as np
gm_map = pd.read_csv("export_data/saturn_vs_uce_cell_type_maps - green_monkey_lung.csv").set_index("human cell_type").to_dict()['green_monkey cell_type']
chh_map = pd.read_csv("export_data/saturn_vs_uce_cell_type_maps - chicken_heart.csv").set_index("human cell_type").to_dict()['chicken cell_type']
chr_map = pd.read_csv("export_data/saturn_vs_uce_cell_type_maps - chicken_retina.csv").set_index("human cell_type").to_dict()['chicken cell_type']
nms_map = pd.read_csv("export_data/saturn_vs_uce_cell_type_maps - nmr_spleen.csv").set_index("human cell_type").to_dict()['nmr cell_type']

for d in [gm_map, chh_map, chr_map, nms_map]:
    for k, v in d.items():
        if type(v) != str and np.isnan(v):
            d[k] = k


green_monkey_lung_saturn_ad.obs["cell_type_mapped"] = [gm_map.get(s, s) for s in green_monkey_lung_saturn_ad.obs["labels2"]]
chicken_heart_saturn_ad.obs["cell_type_mapped"] = [chh_map.get(s, s.replace("-1", "").replace("-2", "").replace("MT-enriched c", "C")) for s in chicken_heart_saturn_ad.obs["labels2"]]
chicken_retina_saturn_ad.obs["cell_type_mapped"] = [chr_map.get(s, s) for s in chicken_retina_saturn_ad.obs["labels2"]]
nmr_spleen_saturn_ad.obs["cell_type_mapped"] = [nms_map.get(s, s.replace(".1", "").replace(".2", "").replace(".3", "").replace(".4", "").replace("Red-pulp-m", "M").replace("Ltf-high-neutrophil", "Neutrophil")) for s in nmr_spleen_saturn_ad.obs["labels2"]]


# Load in UCE AnnDatas

In [7]:
green_monkey_lung_uce_ad = sc.read("export_data/green_monkey_covid_uce_adata.h5ad")
chicken_heart_uce_ad = sc.read("export_data/chicken_heart_uce_adata.h5ad")
chicken_retina_uce_ad = sc.read("export_data/chicken_uce_adata.h5ad")
nmr_spleen_uce_ad = sc.read("export_data/nmr_female_uce_adata.h5ad")

In [8]:
tabula_heart_uce_ad = sc.read("export_data/tabula_sapiens_heart_uce_adata.h5ad")
tabula_lung_uce_ad = sc.read("export_data/tabula_sapiens_lung_uce_adata.h5ad")
tabula_spleen_uce_ad = sc.read("export_data/tabula_sapiens_spleen_uce_adata.h5ad")
retina_uce_ad = sc.read("export_data/human_retina_uce_adata.h5ad")

In [9]:
import anndata
green_monkey_lung_uce_ad = anndata.concat({"green_monkey":green_monkey_lung_uce_ad, "human":tabula_lung_uce_ad}, label="species")
chicken_heart_uce_ad = anndata.concat({"chicken":chicken_heart_uce_ad, "human":tabula_heart_uce_ad}, label="species")
chicken_retina_uce_ad = anndata.concat({"chicken":chicken_retina_uce_ad, "human":retina_uce_ad}, label="species")
nmr_spleen_uce_ad = anndata.concat({"nmr":nmr_spleen_uce_ad, "human":tabula_spleen_uce_ad}, label="species")

/tmp/user/21290/ipykernel_728498/717338908.py:2: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  green_monkey_lung_uce_ad = anndata.concat({"green_monkey":green_monkey_lung_uce_ad, "human":tabula_lung_uce_ad}, label="species")
/tmp/user/21290/ipykernel_728498/717338908.py:3: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  chicken_heart_uce_ad = anndata.concat({"chicken":chicken_heart_uce_ad, "human":tabula_heart_uce_ad}, label="species")
/tmp/user/21290/ipykernel_728498/717338908.py:4: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  chicken_retina_uce_ad = anndata.concat({"chicken":chicken_retina_uce_ad, "human":retina_uce_ad}, label="species")
/tmp/user/21290/ipykernel_728498/717338908.py:5: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  nmr_spleen_uce_ad = anndata.concat({"n

# Centroid Calculation UCE

In [10]:
green_monkey_lung_uce_ad.obs["cell_type_mapped"] = [gm_map.get(s, s) for s in green_monkey_lung_uce_ad.obs["cell_type"]]
chicken_heart_uce_ad.obs["cell_type_mapped"] = [chh_map.get(s, s.replace("-1", "").replace("-2", "").replace("MT-enriched c", "C")) for s in chicken_heart_uce_ad.obs["cell_type"]]
chicken_retina_uce_ad.obs["cell_type_mapped"] = [chr_map.get(s, s) for s in chicken_retina_uce_ad.obs["cell_type"]]
nmr_spleen_uce_ad.obs["cell_type_mapped"] = [nms_map.get(s, s.replace(".1", "").replace(".2", "").replace(".3", "").replace(".4", "").replace("Red-pulp-m", "M").replace("Ltf-high-neutrophil", "Neutrophil")) for s in nmr_spleen_uce_ad.obs["cell_type"]]

# Read in SAMAP

In [11]:
green_monkey_lung_samap = sc.read("export_data/hu_gm_lung_0.h5ad")
chicken_heart_samap = sc.read("export_data/hu_ck_heart_0.h5ad")
chicken_retina_samap = sc.read("export_data/hu_ck_retina_0.h5ad")
nmr_spleen_samap = sc.read("export_data/hu_nmrf_spleen_0.h5ad")

In [12]:
green_monkey_lung_samap.obs["cell_type_mapped"] = [gm_map.get(s, s) for s in green_monkey_lung_samap.obs["cell_type"]]
chicken_heart_samap.obs["cell_type_mapped"] = [chh_map.get(s, s.replace("-1", "").replace("-2", "").replace("MT-enriched c", "C")) for s in chicken_heart_samap.obs["cell_type"]]
chicken_retina_samap.obs["cell_type_mapped"] = [chr_map.get(s, s) for s in chicken_retina_samap.obs["cell_type"]]
nmr_spleen_samap.obs["cell_type_mapped"] = [nms_map.get(s, s.replace(".1", "").replace(".2", "").replace(".3", "").replace(".4", "").replace("Red-pulp-m", "M").replace("Ltf-high-neutrophil", "Neutrophil")) for s in nmr_spleen_samap.obs["cell_type"]]

In [13]:
green_monkey_lung_uce_ad.obsm["X_saturn"] = green_monkey_lung_saturn_ad[green_monkey_lung_uce_ad.obs_names].X
green_monkey_lung_uce_ad.obsm["X_samap"] = green_monkey_lung_samap[green_monkey_lung_uce_ad.obs_names].X

green_monkey_lung_uce_ad

AnnData object with n_obs × n_vars = 83977 × 13091
    obs: 'tissue', 'cell_type', 'n_genes', 'species', 'cell_type_mapped'
    obsm: 'X_uce', 'X_saturn', 'X_samap'

In [14]:
cell_type_species_counts = green_monkey_lung_uce_ad.obs[["species", "cell_type_mapped"]].groupby("cell_type_mapped").agg(lambda x: len(np.unique(x)))
keep_common_cts = list(cell_type_species_counts[cell_type_species_counts["species"] == 2].index)
keep_common_cts

['B Cell',
 'Cilated',
 'DC',
 'Fibroblasts',
 'Macrophage',
 'Monocyte',
 'NK Cell',
 'Neutrophil',
 'Plasma Cell',
 'Pneumocyte',
 'T Cell',
 'Vessel']

In [15]:
nmr_spleen_saturn_ad.obs_names = nmr_spleen_uce_ad.obs_names

In [16]:
nmr_spleen_uce_ad.obsm["X_saturn"] = nmr_spleen_saturn_ad[nmr_spleen_uce_ad.obs_names].X
nmr_spleen_uce_ad.obsm["X_samap"] = nmr_spleen_samap[nmr_spleen_uce_ad.obs_names].X

nmr_spleen_uce_ad

AnnData object with n_obs × n_vars = 75058 × 1
    obs: 'sex', 'cell_type', 'n_genes', 'species', 'cell_type_mapped'
    obsm: 'X_uce', 'X_saturn', 'X_samap'

In [17]:
cell_type_species_counts = nmr_spleen_uce_ad.obs[["species", "cell_type_mapped"]].groupby("cell_type_mapped").agg(lambda x: len(np.unique(x)))
keep_common_cts = list(cell_type_species_counts[cell_type_species_counts["species"] == 2].index)
keep_common_cts

['B-cell',
 'CD8-T-cell',
 'Dendritic-cell',
 'Erythroid-cell',
 'Macrophage',
 'Monocyte',
 'Naive-T-cell',
 'Neutrophil',
 'Plasma-cell',
 'T-cell',
 'Timeless-high-dendritic-cell']

In [18]:
chicken_heart_uce_ad.obsm["X_saturn"] = chicken_heart_saturn_ad[chicken_heart_uce_ad.obs_names].X
chicken_heart_uce_ad.obsm["X_samap"] = chicken_heart_samap[chicken_heart_uce_ad.obs_names].X

chicken_heart_uce_ad

AnnData object with n_obs × n_vars = 33820 × 9479
    obs: 'cell_type', 'n_genes', 'species', 'cell_type_mapped'
    obsm: 'X_uce', 'X_saturn', 'X_samap'

In [19]:
cell_type_species_counts = chicken_heart_uce_ad.obs[["species", "cell_type_mapped"]].groupby("cell_type_mapped").agg(lambda x: len(np.unique(x)))
keep_common_cts = list(cell_type_species_counts[cell_type_species_counts["species"] == 2].index)
keep_common_cts

['Cardiomyocytes',
 'Endocardial cells',
 'Fibroblast cells',
 'Macrophages',
 'Mural cells']

In [20]:
chicken_retina_uce_ad.obsm["X_saturn"] = chicken_retina_saturn_ad[chicken_retina_uce_ad.obs_names].X
chicken_retina_uce_ad.obsm["X_samap"] = chicken_retina_samap[chicken_retina_uce_ad.obs_names].X

chicken_retina_uce_ad

AnnData object with n_obs × n_vars = 134401 × 10770
    obs: 'donor_id', 'disease', 'sex', 'cell_type', 'n_genes', 'species', 'cell_type_mapped'
    obsm: 'X_uce', 'X_saturn', 'X_samap'

In [21]:
cell_type_species_counts = chicken_retina_uce_ad.obs[["species", "cell_type_mapped"]].groupby("cell_type_mapped").agg(lambda x: len(np.unique(x)))
keep_common_cts = list(cell_type_species_counts[cell_type_species_counts["species"] == 2].index)
keep_common_cts

['Amacrine',
 'Bipolar',
 'Cones',
 'Horizontal',
 'Muller Glia',
 'Retinal Ganglion',
 'Rods']

# Logistic Regression Scores

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
ads_to_name = {
    "gm_lung":green_monkey_lung_uce_ad, 
    "ch_heart":chicken_heart_uce_ad, 
    "ch_retina":chicken_retina_uce_ad, 
    "nmr_spleen":nmr_spleen_uce_ad
}

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
for species, ad in ads_to_name.items():
    emb = "X_uce"
    human_ad = ad[ad.obs.species == "human"]
    species_ad = ad[ad.obs.species != "human"]

    # make sure mapping is possible
    poss_cts = species_ad.obs["cell_type_mapped"].unique()
    human_ad = human_ad[human_ad.obs["cell_type_mapped"].isin(poss_cts)]

    human_ad_poss_cts = human_ad.obs["cell_type_mapped"].unique()
    # calculate maximum value
    max_prop_poss = np.mean(species_ad.obs["cell_type_mapped"].isin(human_ad_poss_cts))
   
    X, Y = human_ad.obsm[emb], human_ad.obs["cell_type_mapped"]
    clf = LogisticRegression(random_state=0, n_jobs=24).fit(X, Y)
    #print(f"UCE Train Score: {clf.score(X, Y)}")
    uce_preds = clf.predict(species_ad.obsm[emb])
    

    emb = "X_saturn"
    X, Y = human_ad.obsm[emb], human_ad.obs["cell_type_mapped"]
    clf = LogisticRegression(random_state=0, n_jobs=24).fit(X, Y)
    saturn_preds = clf.predict(species_ad.obsm[emb])
    
    #print(f"SATURN Train Score: {clf.score(X, Y)}")
    

    emb = "X_samap"
    X, Y = human_ad.obsm[emb], human_ad.obs["cell_type_mapped"]
    clf = LogisticRegression(random_state=0, n_jobs=24).fit(X, Y)
    samap_preds = clf.predict(species_ad.obsm[emb])
    
    #print(f"SAMAP Train Score: {clf.score(X, Y)}")
    print(species)
    print(f"Max Value: {max_prop_poss}")
    print(f"UCE Test Score: {accuracy_score(y_pred=uce_preds, y_true=species_ad.obs['cell_type_mapped'])}")
    print(f"SATURN Test Score: {accuracy_score(y_pred=saturn_preds, y_true=species_ad.obs['cell_type_mapped'])}")
    print(f"SAMAP Test Score: {accuracy_score(y_pred=samap_preds, y_true=species_ad.obs['cell_type_mapped'])}")
    print("--------------------\n\n")

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


gm_lung
Max Value: 0.961258929495807
UCE Test Score: 0.7658556786416814
SATURN Test Score: 0.9449218345584429
SAMAP Test Score: 0.6810435862925769
--------------------




/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ch_heart
Max Value: 0.5524086937037866
UCE Test Score: 0.32977817611472104
SATURN Test Score: 0.2995294644857719
SAMAP Test Score: 0.4637239524983195
--------------------




/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ch_retina
Max Value: 0.9952250625982647
UCE Test Score: 0.761078437081465
SATURN Test Score: 0.2840214289873639
SAMAP Test Score: 0.5653642345542421
--------------------




/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


nmr_spleen
Max Value: 0.9872119647293808
UCE Test Score: 0.6518975008525357
SATURN Test Score: 0.12266770594826326
SAMAP Test Score: 0.6397671359672626
--------------------




/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/envs/faiss_1.8.0/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
